# Сравнение выручки на пользователя

# Введение

Для моделирования выручки на пользователя применимы распределение Парето и лог-нормальное распределение.  

https://en.wikipedia.org/wiki/Pareto_distribution  
https://en.wikipedia.org/wiki/Log-normal_distribution  
https://arxiv.org/abs/cond-mat/0412004  
https://academic.oup.com/bioscience/article/51/5/341/243981?login=false  


https://en.wikipedia.org/wiki/Gibrat%27s_law

Критерием выбора может быть ограничение минимальной суммы заказа: Парето если сумма ограничена, логнормальное если может быть 0 [Mitz].


In [ ]:
import pandas as pd
import numpy as np
np.random.seed(7)

from collections import namedtuple

import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#todo: update scipy; make venv

# Лог-нормальное

Лог-нормальное распределение  
https://en.wikipedia.org/wiki/Log-normal_distribution  
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.lognorm.html  

$$
X \sim Lognormal(\mu, \sigma^2)
\\
\ln(X) \sim Norm(\mu, \sigma^2)
\\
f(x) = \frac{1}{x \sigma \sqrt{2 \pi}} e^{-\dfrac{(\ln(x) - \mu)^2}{2 \sigma^2}}
\\
E[x] = e^{\left(\mu + \sigma^2/2\right)}
$$

In [ ]:
x = np.linspace(0, 10, 2000)
fig = go.Figure()
for sigma, loc, scale in [(1, 0, 1), (2, 0, 1), (1, 0, 2)]:
    fig.add_trace(go.Scatter(x=x, y=stats.lognorm.pdf(x, s=sigma, loc=loc, scale=scale), 
                             mode='lines', 
                             name=f's={sigma}, loc={loc}, scale={scale}'))
fig.update_layout(title='Lognormal',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

В байесовском подходе для оценки плотности вероятности параметров модели используется соотношение

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)}
$$

Сопряженное априорное распределение - произведение нормального и гамма-распределений аналогично сопряженному распределению для нормального распределения [ConjPrior](https://en.wikipedia.org/wiki/Conjugate_prior), [compendum of priors], [GammaDist](https://en.wikipedia.org/wiki/Gamma_distribution), [SciPyStatsGamma](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gamma.html)

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)} \propto P(data | model) P(model)
$$

$$
P(data | model) = P(x | \mu, \sigma) = 
\frac{1}{x \sigma \sqrt{2 \pi}} e^{-\dfrac{(\ln(x) - \mu)^2}{2 \sigma^2}}
$$

$$
P(model) = 
Norm(\mu | \tau; \mu_0, k_0) Gamma(\tau; a, b) = 
(\tau k_0/ \pi)^{1/2} e^{- \tau k_0 (\mu-\mu_0)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}, \quad \tau>0, \quad a,b>0 .
$$

Для $N$ точек:

$$
\begin{split}
P(model | data) 
& \propto
P(data | model) P(model) 
\\
& \propto
\prod_i^N \left( \frac{\tau}{x_i \pi} \right)^{1/2} e^{- \tau (\ln(x_i)-\mu)^2}
\left( \frac{\tau k_0}{\pi} \right)^{1/2} e^{- \tau k_0 (\mu-\mu_0)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}
\\
& \propto
\tau^{1/2} e^{- \tau k_N (\mu-\mu_N)^2}
\tau^{a_N-1} e^{-b_N \tau}
\\
& = Norm(\mu | \tau; \mu_N, k_N) Gamma(\tau; a_N, b_N)
\end{split}
$$

$$
\begin{split}
a_N & = a + N/2
\\
k_N & = k_0 + N
\\
\mu_N & = \frac{k_0 \mu_0 + \sum_i^N \ln x_i}{k_0 + N}
\\
b_N & = b + k_0 \mu_0^2 - k_N \mu_N^2 + \sum_i^N (\ln x_i )^2
\end{split}
$$

Исходные значения параметров $\mu_0$ и $\sigma_0$ можно задать произвольно. Но если они будут далеки
от реальных, то модель может оказаться неточной. Поэтому удобнее задавать их либо
по историческим данным, либо на основе части данных из сэмпла. Для $\mu_0$ можно
выбрать среднее $\bar{x}_1$ в первом наборе из $n_{split}$ точек, для $\sigma_0^2$ - дисперсию в этой
части сэмпла $\sigma_{\bar{x}_1}^2$, отнесенную к количеству точек $n_{split}$.
Для начальных значений $a_0$ и $b_0$ можно выбрать 2 и 1, поскольку при таких значениях
параметров мода $p(\lambda) = Gamma(\lambda, a=2, b=1)$ будет равна 1. Значение
$k_0$ можно выбрать $1/25$, чтобы вначале обеспечить широкую область для $P(\mu) = N(\mu_0, (5 \sigma_0)^2)$:

$$
\mu_0 = \bar{x}_1, \sigma_0 = \sigma_{\bar{x}_1} / \sqrt{n_{split}} ,
\\ 
k_0 = 1/25 ,
\\
a_0 = 2, b_0 = 1 .
$$

In [ ]:
ConjugateLogNormalParams = namedtuple('ConjugateLogNormalParams', 'mu k a b')

def initial_parameters(mu, k=1/25, a=2, b=1):
    return ConjugateLogNormalParams(mu=mu, k=k, a=a, b=b)

def compute_posterior_parameters(x, initial_pars):
    lnx = [np.log(xi) for xi in x]
    a_n = initial_pars.a + len(x) * 1.0 / 2
    k_n = initial_pars.k + len(x)
    mu_n = (initial_pars.k * initial_pars.mu + np.sum(lnx)) / (initial_pars.k + 1)    
    b_n = initial_pars.b + initial_pars.k * initial_pars.mu**2 - k_n * mu_n**2 + np.sum(lnx**2)
    return ConjugateLogNormalParams(mu=mu_n, k=k_n, a=a_n, b=b_n)

# Парето

Распределение Парето [[ParetoDist](https://en.wikipedia.org/wiki/Pareto_distribution), [SciPyPareto](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pareto.html)] 

$$
P(x; x_m, c) = 
\begin{cases}
\dfrac{c x_m^c}{x^{c + 1}} & x \ge x_m
\\
0 & x < x_m.
\end{cases}
$$

In [ ]:
x = np.linspace(0, 10, 2000)
fig = go.Figure()
for b, loc, scale in [(1, 0, 1), (2, 0, 1), (3, 0, 4), (4, 1, 1)]:
    fig.add_trace(go.Scatter(x=x, y=stats.pareto.pdf(x, b=b, loc=loc, scale=scale), 
                             mode='lines', 
                             name=f'b={b}, loc={loc}, x_m={scale}'))
fig.update_layout(title='Pareto',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

В байесовском подходе для оценки плотности вероятности параметров модели используется соотношение

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)}.
$$

Сопряженное априорное распределение - Гамма-распределение
https://en.wikipedia.org/wiki/Conjugate_prior ,
[ссылка на a compendum of priors]   


$$
P(model) = Gamma(x; a, b) = \frac{b^a}{\Gamma(a)} x^{a-1} e^{-b x}, \quad x>0, \quad a,b>0 .
$$

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)} \propto P(data | model) P(model)
$$

$$
P(data | model) = P(x | \alpha) = 
\begin{cases}
\dfrac{\alpha}{x_m} \left(\dfrac{x}{x_m}\right)^{-(\alpha+1)} & x \ge x_m
\\
0 & x < x_m.
\end{cases}
$$

$$
P(model) = Gamma(\alpha; a, b) = \frac{b^a}{\Gamma(a)} \alpha^{a-1} e^{-b \alpha}, \quad \alpha>0, \quad a,b>0 .
$$

$$
\begin{split}
P(model | data) 
& \propto
\frac{\alpha}{x_m}\left(\frac{x}{x_m}\right)^{-(\alpha + 1)} \frac{b^a}{\Gamma(a)} \alpha^{a-1} e^{-b \alpha}
\\
& \propto 
\alpha e^{-(\alpha+1) \ln(x/x_m)} \alpha^{a-1} e^{-b \alpha}
\\
& \propto
\alpha^{a} e^{-\alpha (b + \ln(x/x_m))}
\\
& \sim
Gamma(\alpha; a+1, b + \ln(x/x_m))
\end{split}
$$

Для N точек
$$
P(model | data) = Gamma(\alpha; a+N, b + \sum_i^N\ln(x_i/x_m))
$$

In [ ]:
x = np.linspace(0, 10, 2000)
fig = go.Figure()
for a,b in [(1,1), (2,1), (3,1), (9, 3), (45, 15), (90, 30)]:
    fig.add_trace(go.Scatter(x=x, y=stats.gamma.pdf(x, a=a, scale=1/b), mode='lines', name=f'a={a}, b={b}'))
fig.update_layout(title='Gamma Distribution',
                  xaxis_title='$\lambda$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

# Оценка формы распределения Парето

In [ ]:
def posterior_params(alpha, beta, x_m, data):
    alpha_post = alpha + len(data)
    beta_post = beta + np.sum([np.log(x/x_m) for x in data])
    return alpha_post, beta_post

def posterior_dist(alpha, beta, x_m, data):
    alpha_post = alpha + len(data)
    beta_post = beta + np.sum([np.log(x/x_m) for x in data])
    dist = stats.gamma(a=alpha_post, scale=1/beta_post)
    return dist

b = 2.8
scale = 1
nsample = 1000

exact_dist = stats.pareto(b=b)

data = exact_dist.rvs(nsample)

alpha_prior = 2
beta_prior = 1

postdist = posterior_dist(alpha=alpha_prior, beta=beta_prior, x_m=scale, data=data)
#postdist

x = np.linspace(0, 10, 2000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=postdist.pdf(x), mode='lines', name=f'Posterior'))
fig.update_layout(title='Gamma Distribution',
                  xaxis_title='$\lambda$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

Апостериорное распределение

Среднее

$$
E[x] = 
\begin{cases}
\dfrac{\alpha x_m}{\alpha - 1} & \alpha > 1
\\
\infty & \alpha \le 1
\end{cases}
$$

Медиана

$$
m = x_m 2^{1/\alpha}
$$

In [ ]:
alpha_nsamp = 50000
samp = postdist.rvs(alpha_nsamp)
samp_mean = samp * scale / (samp-1)
samp_median = scale * 2**(1/samp)

fig = go.Figure()
fig.add_trace(go.Histogram(x=samp_mean, histnorm='probability density', name='Mean Hist', nbinsx=100))
fig.add_vline(exact_dist.mean(), name='Original Distribution Mean')
fig.show()

fig = go.Figure()
fig.add_trace(go.Histogram(x=samp_median, histnorm='probability density', name='Median Hist', nbinsx=100))
fig.add_vline(exact_dist.median(), name='Original Distribution Median')
fig.show()

# Сравнение распределений

In [ ]:
# params = {
#     'A': {'b': 2.8}
#     'B': {'b': 2.9}
# }

b = 1.3
scale = 1

exact_dist_a = stats.pareto(b=b)
exact_dist_b = stats.pareto(b=b*1.05)

nsample = 1000

samp_a = exact_dist_a.rvs(nsample)
samp_b = exact_dist_b.rvs(nsample)

alpha_prior = 2
beta_prior = 1

postdist_a = posterior_dist(alpha=alpha_prior, beta=beta_prior, x_m=scale, data=samp_a)
postdist_b = posterior_dist(alpha=alpha_prior, beta=beta_prior, x_m=scale, data=samp_b)
#postdist

x = np.linspace(0, 5, 2000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=postdist_a.pdf(x), mode='lines', name=f'A'))
fig.add_trace(go.Scatter(x=x, y=postdist_b.pdf(x), mode='lines', name=f'B'))
fig.update_layout(title='Posterior Distribution',
                  xaxis_title='$alpha$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()


post_nsamp = 50000
post_samp_a = postdist_a.rvs(alpha_nsamp)
post_samp_b = postdist_b.rvs(alpha_nsamp)
p_alpha_b_gt_alpha_a = np.sum(post_samp_b > post_samp_a) / post_nsamp
print(f'P(alpha_B > alpha_A) = {p_alpha_b_gt_alpha_a :.2f}')

# Ссылки

Fink, Daniel (1997). "A Compendium of Conjugate Priors"
https://courses.physics.ucsd.edu/2018/Fall/physics210b/REFERENCES/conjugate_priors.pdf  

Internet Mathematics Vol. 1, No. 2: 226-251  
A Brief History of Generative Models for Power Law and Lognormal Distributions  
Michael Mitzenmacher  
http://www.eecs.harvard.edu/~michaelm/postscripts/im2004a.pdf

# Приложение

## Сопряженное распределение к нормальному

Сопряженное априорное распределение для нормальной функции правдоподобия:

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)} \propto P(data | model) P(model)
$$

$$
P(data | model) = P(x | \mu, \sigma_x) = 
\frac{1}{\sqrt{2 \pi \sigma_x^2}} e^{-\dfrac{(x - \mu)^2}{2 \sigma_x^2}}
$$

Один параметр: $\mu$ меняется, $\sigma$ фиксировано.  
Сопряженное априорное распределение - нормальное распределение.

$$
P(model) = Norm(\mu | \mu_0, \sigma_0) = 
\frac{1}{\sqrt{2 \pi \sigma_{0}^2}} e^{- \dfrac{(\mu-\mu_0)^2}{2 \sigma_{0}^2}} 
$$

$$
\begin{split}
P(model | data) 
& \propto
\frac{1}{\sqrt{2 \pi \sigma_x^2}} e^{-\dfrac{(x - \mu)^2}{2 \sigma_x^2}}
\frac{1}{\sqrt{2 \pi \sigma_{0}^2}} e^{- \dfrac{(\mu-\mu_0)^2}{2 \sigma_{0}^2}} 
& \propto
e^{-\dfrac{(\mu - \mu_1)^2}{2 \sigma_1^2}}
= Norm(\mu | \mu_1, \sigma_1)
\end{split}
$$

$$
\dfrac{x^2 - 2 \mu x + \mu^2}{2 \sigma_x^2} + \dfrac{\mu^2 - 2\mu \mu_0 + \mu_0^2}{2 \sigma_{0}^2}
=
\mu^2 \left[ \frac{1}{2 \sigma_x^2} + \frac{1}{2 \sigma_0^2} \right] -
2 \mu \left[ \frac{x}{2 \sigma_x^2} + \frac{\mu_0}{2 \sigma_{0}^2} \right]  + C
=
\dfrac{\mu^2 - 2 \mu \mu_1 + \mu_1^2}{2 \sigma_1^2} - \frac{\mu_1^2}{2 \sigma_1^2} + C
$$

$$
\frac{1}{2 \sigma_1^2} = \left[ \frac{1}{2 \sigma_x^2} + \frac{1}{2 \sigma_0^2} \right],
\quad
\frac{\mu_1}{2 \sigma_1^2} = \left[ \frac{x}{2 \sigma_x^2} + \frac{\mu_0}{2 \sigma_{0}^2} \right]
\\
\sigma_1^2 = \frac{\sigma_0^2 \sigma_x^2}{\sigma_0^2 + \sigma_x^2},
\quad
\mu_1 = \frac{x \sigma_0^2 + \mu_0 \sigma_x^2}{\sigma_0^2 + \sigma_x^2}
$$

Для $N$ точек:

$$
\begin{split}
P(model | data) 
& \propto
\prod_i^N
\frac{1}{\sqrt{2 \pi \sigma_x^2}} e^{-\dfrac{(x_i - \mu)^2}{2 \sigma_x^2}}
\frac{1}{\sqrt{2 \pi \sigma_{0}^2}} e^{- \dfrac{(\mu-\mu_0)^2}{2 \sigma_{0}^2}} 
& \propto
e^{-\dfrac{(\mu - \mu_N)^2}{2 \sigma_N^2}}
= Norm(\mu | \mu_N, \sigma_N)
\end{split}
$$

$$
\dfrac{\mu^2 - 2\mu \mu_0 + \mu_0^2}{2 \sigma_{0}^2} + \sum_i^N \dfrac{x_i^2 - 2 \mu x_i + \mu^2}{2 \sigma_x^2}
=
\mu^2 \left[ \frac{1}{2 \sigma_0^2} + \frac{N}{2 \sigma_x^2} \right] -
2 \mu \left[ \frac{\mu_0}{2 \sigma_{0}^2} + \sum_i^N \frac{x_i}{2 \sigma_x^2} \right]  + C
=
\dfrac{\mu^2 - 2 \mu \mu_N + \mu_N^2}{2 \sigma_N^2} - \frac{\mu_N^2}{2 \sigma_N^2} + C
$$

$$
\frac{1}{2 \sigma_N^2} = \left[ \frac{1}{2 \sigma_0^2} + \frac{N}{2 \sigma_x^2} \right],
\quad
\frac{\mu_N}{2 \sigma_N^2} = \left[ \frac{\mu_0}{2 \sigma_{0}^2} + \sum_i^N \frac{x_i}{2 \sigma_x^2} \right]
\\
\sigma_N^2 = \frac{\sigma_0^2 \sigma_x^2}{\sigma_x^2 + N \sigma_0^2},
\quad
\mu_N = \frac{\mu_0 \sigma_x^2 + \sigma_0^2 \sum_i^N x_i}{\sigma_x^2 + N \sigma_0^2 }
$$

Один параметр: $\sigma$ меняется, $\mu$ фиксировано.  
Сопряженное априорное распределение - гамма-распределение.

$$
P(model) = Gamma(\tau; a, b) = \frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}, \quad \tau>0, \quad a,b>0 .
$$

$$
\begin{split}
P(model | data) 
& \propto
\frac{\tau^{1/2}}{\sqrt{\pi}} e^{-\tau (x - \mu_x)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}
& \propto
\tau^{a-1+1/2} e^{-b \tau - \tau (x - \mu_x)^2 }
= Gamma(\tau; a+1/2, b + (x - \mu_x)^2)
\end{split}
$$

Для $N$ точек:

$$
\begin{split}
P(model | data) 
& \propto
\prod_i^N
\frac{\tau^{1/2}}{\sqrt{\pi}} e^{-\tau (x_i - \mu_x)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}
& \propto
\tau^{a-1+N/2} e^{-b \tau - \tau \sum_i^N (x_i - \mu_x)^2 }
= Gamma(\tau; a+N/2, b + \sum_i^N (x_i - \mu_x)^2)
\end{split}
$$

Два параметра: $\mu, \sigma$.  

$$
P(data | model) = P(x | \mu, \sigma_x) = 
\frac{1}{\sqrt{2 \pi \sigma_x^2}} e^{-\dfrac{(x - \mu)^2}{2 \sigma_x^2}}
$$

Сопряженное априорное распределение - произведение нормального и гамма-распределений.

$$
P(model) = 
Norm(\mu | \tau; \mu_0, k_0) Gamma(\tau; a, b) = 
(\tau k_0/ \pi)^{1/2} e^{- \tau k_0 (\mu-\mu_0)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}, \quad \tau>0, \quad a,b>0 .
$$


$$
\begin{split}
P(model | data) 
& \propto
P(data | model) P(model) 
\\
& \propto
(\tau/ \pi)^{1/2} e^{- \tau (x-\mu)^2}
(\tau k_0 / \pi)^{1/2} e^{- \tau k_0 (\mu-\mu_0)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}
\\
& \propto
\tau^{1/2} e^{- \tau k_1 (\mu-\mu_1)^2}
\tau^{a_1-1} e^{-b_1 \tau}
\\
& = Norm(\mu | \tau; \mu_1, k_1) Gamma(\tau; a_1, b_1)
\end{split}
$$

$$
a_1 = a + 1/2
$$

$$
(x-\mu)^2 + k_0 (\mu-\mu_0)^2 + b =
\\
\mu^2 (k_0 + 1) - 2 \mu (x + k_0 \mu_0) + x^2 + k_0 \mu_0^2 + b = 
\\
(k_0 + 1) \left[ \mu^2 - 2 \mu \frac{x + k_0 \mu_0}{k_0 + 1} + \left(\frac{x + k_0 \mu_0}{k_0 + 1}\right)^2 \right]
- \frac{(x + k_0 \mu_0)^2}{k_0 + 1} + x^2 + k_0 \mu_0^2 + b
\\
= k_1 (\mu-\mu_1)^2 + b_1
$$

$$
k_1 = k_0 + 1
\\
\mu_1 = \frac{x + k_0 \mu_0}{k_0 + 1}
\\
b_1 = b + x^2 + k_0 \mu_0^2 - \frac{(x + k_0 \mu_0)^2}{k_0 + 1}
\\
= b + k_0 \mu_0^2 - k_1 \mu_1^2 + x^2
$$

$$
b_1 = b + \frac{x^2 k_0 + x^2 + k_0^2 \mu_0^2 + k_0 \mu_0^2 - x^2 - 2xk_0\mu_0 - k_0^2 \mu_0^2}{k_0+1}
= b + \frac{x^2 k_0 + k_0 \mu_0^2 - 2xk_0\mu_0}{k_0+1}
\\
= b + \frac{k_0}{k_0+1}(x - \mu_0)^2
$$

Для $N$ точек:

$$
\begin{split}
P(model | data) 
& \propto
P(data | model) P(model) 
\\
& \propto
\prod_i^N (\tau/ \pi)^{1/2} e^{- \tau (x_i-\mu)^2}
(\tau k_0 / \pi)^{1/2} e^{- \tau k_0 (\mu-\mu_0)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}
\\
& \propto
\tau^{1/2} e^{- \tau k_N (\mu-\mu_N)^2}
\tau^{a_N-1} e^{-b_N \tau}
\\
& = Norm(\mu | \tau; \mu_N, k_N) Gamma(\tau; a_N, b_N)
\end{split}
$$

$$
a_N = a + N/2
$$

$$
\sum_i^N (x_i - \mu)^2 + k_0 (\mu-\mu_0)^2 + b =
\\
\mu^2 (k_0 + N) - 2 \mu (k_0 \mu_0 + \sum_i^N x_i) + \sum_i^N x_i^2 + k_0 \mu_0^2 + b = 
\\
(k_0 + N) \left[ \mu^2 - 2 \mu \frac{k_0 \mu_0 + \sum_i^N x_i}{k_0 + N} + 
\left(\frac{k_0 \mu_0 + \sum_i^N x_i}{k_0 + N}\right)^2 \right]
- \frac{(k_0 \mu_0 + \sum_i^N x_i)^2}{k_0 + N} + \sum_i^N x_i^2 + k_0 \mu_0^2 + b
\\
= k_N (\mu-\mu_N)^2 + b_N
$$

$$
k_N = k_0 + N
\\
\mu_N = \frac{k_0 \mu_0 + \sum_i^N x_i}{k_0 + N}
\\
b_N = b + \sum_i^N x_i^2 + k_0 \mu_0^2 - \frac{(k_0 \mu_0 + \sum_i^N x_i)^2}{k_0 + N}
\\
b_N = b + k_0 \mu_0^2 - k_N \mu_N^2 + \sum_i^N x_i^2
$$

## Лог-нормальное

$$
P(data | model) = P(x | \mu, \sigma) = 
\frac{1}{x \sigma \sqrt{2 \pi}} e^{-\dfrac{(\ln(x) - \mu)^2}{2 \sigma^2}}
$$

$$
\begin{split}
P(model | data) 
& \propto
P(data | model) P(model) 
\\
& \propto
(\tau/ x \pi)^{1/2} e^{- \tau (\ln(x)-\mu)^2}
(\tau k_0 / \pi)^{1/2} e^{- \tau k_0 (\mu-\mu_0)^2}
\frac{b^a}{\Gamma(a)} \tau^{a-1} e^{-b \tau}
\\
& \propto
\tau^{1/2} e^{- \tau k_1 (\mu-\mu_1)^2}
\tau^{a_1-1} e^{-b_1 \tau}
\\
& = Norm(\mu | \tau; \mu_1, k_1) Gamma(\tau; a_1, b_1)
\end{split}
$$

$$
a_1 = a + 1/2
\\
k_1 = k_0 + 1
\\
\mu_1 = \frac{k_0 \mu_0 + \ln(x)}{k_0 + 1}
\\
b_1 = b + k_0 \mu_0^2 - k_1 \mu_1^2 + (\ln x)^2
$$

## Ломакс

Плотность распределения

$$
p(x) = {\alpha \over \lambda} \left[{1 + {x \over \lambda}}\right]^{-(\alpha+1)}, \qquad x \geq 0
$$

$$
p(x) = {{\alpha\lambda^\alpha} \over {(x + \lambda)^{\alpha+1}}}
$$

Для распределения на пользователей: обезразмерить на $x_m$.  
Потом Ломакс с $\lambda=1$.  
Проще сразу безразмерный вариант распределения.

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)} \propto P(data | model) P(model)
$$

$$
P(data | model) = P(x | \alpha) = {\alpha \over \lambda} \left[{1 + {x \over \lambda}}\right]^{-(\alpha+1)}
= \frac{\alpha}{\lambda}\frac{1}{\left[{1 + (x/\lambda)}\right]^{(\alpha+1)}}
$$

$$
P(model) = Gamma(\alpha; a, b) = \frac{b^a}{\Gamma(a)} \alpha^{a-1} e^{-b \alpha}, \quad \alpha>0, \quad a,b>0 .
$$

$$
\begin{split}
P(model | data) 
& \propto
{\alpha \over \lambda} \left[{1 + {x \over \lambda}}\right]^{-(\alpha+1)} \frac{b^a}{\Gamma(a)} \alpha^{a-1} e^{-b \alpha}
\\
& \propto 
\alpha e^{-(\alpha+1) \ln(1 + x/\lambda)} \alpha^{a-1} e^{-b \alpha}
\\
& \propto
\alpha^{a} e^{-\alpha (b + \ln(1 + x/\lambda))}
\\
& \sim
Gamma(\alpha; a+1, b + \ln(1 + x/\lambda))
\end{split}
$$

Для N точек
$$
P(model | data) = Gamma(\alpha; a+N, b + \sum_i^N\ln(1 + x_i/\lambda))
$$